In [274]:
import pandas as pd
from scipy import stats
from IPython.core.interactiveshell import InteractiveShell
import hvplot.pandas

pd.options.plotting.backend = "holoviews"


InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import panel as pn
import scipy

pn.extension()


In [354]:
df = pd.read_csv("/Users/glebsokolov/projects/noshowappointments.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      110527 non-null  int64  
 1   PatientId       110527 non-null  float64
 2   Gender          110527 non-null  float64
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  float64
 7   Scholarship     110527 non-null  int64  
 8   Hypertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  float64
dtypes: float64(4), int64(8), object(2)
memory usage: 11.8+ MB


In [355]:
df.head()


,Unnamed: 0,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,0,2.987250e+13,0.0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,39.0,0,1,0,0,0,0,0.0
1,1,5.589978e+14,1.0,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,39.0,0,0,0,0,0,0,0.0
2,2,4.262962e+12,0.0,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,45.0,0,0,0,0,0,0,0.0
3,3,8.679512e+11,0.0,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,54.0,0,0,0,0,0,0,0.0
4,4,8.841186e+12,0.0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,39.0,0,1,1,0,0,0,0.0


First, let's preprocess the data: make the target variable boolean, make ordinal
encoding for the Neighborhood and Gender column, cast columns, containing dates to
datetime type, drop AppointmentID (as we wont need it), and I will also calculate a new feature - awaiting time. Then, let's also check the data for outliers and ridiculous records (i.e. negative age and so on)

In [356]:
from sklearn.preprocessing import OrdinalEncoder

del df["Unnamed: 0"]
df[["ScheduledDay", "AppointmentDay"]] = df[["ScheduledDay", "AppointmentDay"]].apply(
    lambda x: pd.to_datetime(x)
)
enc = OrdinalEncoder()
df[["Gender", "Neighbourhood", "No-show"]] = enc.fit_transform(
    df[["Gender", "Neighbourhood", "No-show"]]
)
df["AwaitingTime"] = -(df["ScheduledDay"] - df["AppointmentDay"]).dt.days
df.head()


,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,AwaitingTime
0,2.987250e+13,0.0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,39.0,0,1,0,0,0,0,0.0,0
1,5.589978e+14,1.0,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,39.0,0,0,0,0,0,0,0.0,0
2,4.262962e+12,0.0,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,45.0,0,0,0,0,0,0,0.0,0
3,8.679512e+11,0.0,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,54.0,0,0,0,0,0,0,0.0,0
4,8.841186e+12,0.0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,39.0,0,1,1,0,0,0,0.0,0


In [282]:
df.query("Age==-1 or AwaitingTime<0")
df.sort_values(by="Age", ascending=True)["Age"].unique()[:10]
df = df.drop(index=df.query("Age==-1 or AwaitingTime<0").index)


,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,AwaitingTime
27033,7.839273e+12,1.0,2016-05-10 10:51:53+00:00,2016-05-09 00:00:00+00:00,38,59.0,0,0,0,0,1,0,1.0,-1
55226,7.896294e+12,0.0,2016-05-18 14:50:41+00:00,2016-05-17 00:00:00+00:00,19,69.0,0,0,0,0,1,0,1.0,-1
64175,2.425226e+13,0.0,2016-05-05 13:43:58+00:00,2016-05-04 00:00:00+00:00,22,13.0,0,0,0,0,0,0,1.0,-1
71533,9.982316e+14,0.0,2016-05-11 13:49:20+00:00,2016-05-05 00:00:00+00:00,81,69.0,0,0,0,0,0,0,1.0,-6
72362,3.787482e+12,1.0,2016-05-04 06:50:57+00:00,2016-05-03 00:00:00+00:00,7,78.0,0,0,0,0,0,0,1.0,-1
99832,4.659432e+14,0.0,2016-06-06 08:58:13+00:00,2016-06-06 00:00:00+00:00,-1,60.0,0,0,0,0,0,0,0.0,0


array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8])

As we can see here, we have several recordings with negative awaiting time, we delete them. We also see, that we have a recording with negative age.

Now and further 1 will encode a group who didn't come, and 0 who come.

Now let's view at the distributions of the categorical variables, in order to do so,
let's build bar plots, and  histogram for age, one more bar plot will include all the neighborhoods.

In [6]:
show_opts = [
    1.0,
    0.0,
]
show_opt = pn.widgets.Select(name="No-show", options=show_opts)
idf = df.interactive()
data_pipeline = idf[idf["No-show"] == show_opt]


In [357]:
df.Age.hist()


:Histogram   [Age]   (Age_count)

In [360]:
group = df.groupby(['Neighbourhood', "No-show"]).count()["PatientId"]
group.plot.bar(stacked=True, title='Neighbourhood', rot=90, width=1024) + group.hvplot.table(width=240)

:Layout
   .Bars.PatientId :Bars   [Neighbourhood,No-show]   (PatientId)
   .Table.I        :Table   [Neighbourhood,No-show,PatientId]

Now, lets write a function that will calculate confidence interval for the difference
 of the two proportions (for the show and not-show groups). Then, write a function that will draw a conclusion given the confidence interval.

In [55]:
def proportions_confint_diff(sample1, sample2, alpha=0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.0)
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)

    left_boundary = (p1 - p2) - z * np.sqrt(
        p1 * (1 - p1) / len(sample1) + p2 * (1 - p2) / len(sample2)
    )
    right_boundary = (p1 - p2) + z * np.sqrt(
        p1 * (1 - p1) / len(sample1) + p2 * (1 - p2) / len(sample2)
    )
    return (left_boundary, right_boundary)


def calculate_ci(group, alpha):
    boundaries = proportions_confint_diff(
        df[df[group] == 1]["No-show"], df[df[group] == 0]["No-show"], alpha=alpha
    )
    if boundaries[0] < 0 and boundaries[1] < 0:
        print(f"CI is {boundaries}, proportion in the no-show group tends to be less")
    elif boundaries[0] > 0 and boundaries[1] > 0:
        print(
            f"CI is {boundaries}, proportion in the no-show group tends to be greater"
        )
    else:
        print(f"CI is {boundaries}, can not make definitive decision")


The function below will draw a bar plot for each categorical variable with a table, where a prior probability for show and no-show situations for each subcategory will be calculated

In [346]:
def group_draw(by):
    group = (
        df.groupby([by, "No-show"])
        .count()[["PatientId"]]
        .rename(columns={"PatientId": "Count"})
    )
    ind = group.index
    group = group.reset_index()
    group["ShowProb"] = pd.concat(
        [
            group[group[by] == 0].transform(lambda x: x / sum(x))["Count"],
            group[group[by] == 1].transform(lambda x: x / sum(x))["Count"],
        ]
    )
    group = group[['Count', 'ShowProb']].set_index(ind)
    return group.plot.bar(stacked=True, title=by, rot=90) + group.hvplot.table(
        width=360
    )


In [347]:
group_draw("Scholarship")
calculate_ci("Scholarship", 0.05)


:Layout
   .Bars.I  :Bars   [Scholarship,No-show]   (Count)
   .Table.I :Table   [Scholarship,No-show,Count,ShowProb]

CI is (0.0309542999362394, 0.04770515768787152), proportion in the no-show group tends to be greater


In [348]:
group_draw("Gender")
calculate_ci("Gender", 0.05)


:Layout
   .Bars.I  :Bars   [Gender,No-show]   (Count)
   .Table.I :Table   [Gender,No-show,Count,ShowProb]

CI is (-0.008429305123660698, 0.0014746511919310262), can not make definitive decision


In [349]:
group_draw("Hypertension")
calculate_ci("Hypertension", 0.05)


:Layout
   .Bars.I  :Bars   [Hypertension,No-show]   (Count)
   .Table.I :Table   [Hypertension,No-show,Count,ShowProb]

CI is (-0.041664478829214646, -0.030285528096700872), proportion in the no-show group tends to be less


In [350]:
group_draw("Diabetes")
calculate_ci("Diabetes", 0.05)


:Layout
   .Bars.I  :Bars   [Diabetes,No-show]   (Count)
   .Table.I :Table   [Diabetes,No-show,Count,ShowProb]

CI is (-0.03236014206542362, -0.014757218213109685), proportion in the no-show group tends to be less


In [351]:
group_draw("Alcoholism")
calculate_ci("Alcoholism", 0.05)


:Layout
   .Bars.I  :Bars   [Alcoholism,No-show]   (Count)
   .Table.I :Table   [Alcoholism,No-show,Count,ShowProb]

CI is (-0.014196989031092205, 0.01335089341439441), can not make definitive decision


In [352]:
group_draw("SMS_received")
calculate_ci("SMS_received", 0.05)


:Layout
   .Bars.I  :Bars   [SMS_received,No-show]   (Count)
   .Table.I :Table   [SMS_received,No-show,Count,ShowProb]

CI is (0.10340441820466297, 0.11412680428538112), proportion in the no-show group tends to be greater


Now, let's calculate probabilitites of no-show for each age, neighborhood, scheduled hour and awaiting time and see if there are any patterns

In [149]:
probs_age = pd.crosstab(index=df["Age"], columns=df["No-show"])
probs_age["prob_show"] = probs_age.loc[:, 1] / (
    probs_age.loc[:, 0] + probs_age.loc[:, 1]
)
probs_age.hvplot.scatter(x="Age", y="prob_show")


:Scatter   [Age]   (prob_show)

In [153]:
probs_place = pd.crosstab(index=df["Neighbourhood"], columns=df["No-show"])
probs_place["prob_show"] = probs_place.loc[:, 1] / (
    probs_place.loc[:, 0] + probs_place.loc[:, 1]
)
probs_place.hvplot.scatter(x="Neighbourhood", y="prob_show")


:Scatter   [Neighbourhood]   (prob_show)

As we can see here age's and neighborhood's probabilities are distributed almost uniformly (except for a single peculiar points for each graph)

In [234]:
probs_hour = (
    pd.crosstab(index=df["ScheduledDay"].dt.hour, columns=df["No-show"])
    .reset_index()
    .rename(columns={"ScheduledDay": "ScheduledHour"})
)
probs_hour["prob_hour"] = probs_hour.loc[:, 1] / (
    probs_hour.loc[:, 0] + probs_hour.loc[:, 1]
)
probs_hour.hvplot.scatter(x="ScheduledHour", y="prob_hour")


:Scatter   [ScheduledHour]   (prob_hour)

In [179]:
probs_await = pd.crosstab(index=df["AwaitingTime"], columns=df["No-show"])
probs_await["prob_await"] = probs_await.loc[:, 1] / (
    probs_await.loc[:, 0] + probs_await.loc[:, 1]
)
probs_await.hvplot.scatter(x="AwaitingTime", y="prob_await")


:Scatter   [AwaitingTime]   (prob_await)

In [117]:
# import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
X, y = (
    encoder.fit_transform(
        df.drop(["ScheduledDay", "AppointmentDay", "PatientId", "No-show"], axis=1)
    ),
    df["No-show"],
)


Now let's train and compair a pair of classifiers that will try to predict our target variable.

In [248]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(["ScheduledDay", "AppointmentDay", "PatientId", "No-show"], axis=1),
    df["No-show"],
    test_size=0.2,
)
mclf, bclf = MultinomialNB(), BernoulliNB()
mclf.fit(X_train, y_train); bclf.fit(X_test, y_test)
print("Accuracy:", round(accuracy_score(y_test, mclf.predict(X_test)), 2) * 100, "%")
print("Accuracy:", round(accuracy_score(y_test, bclf.predict(X_test)), 2) * 100, "%")

MultinomialNB()

BernoulliNB()

Accuracy: 71.0 %
Accuracy: 80.0 %


Such a result is excpectable. Bernoulli classifier is well adopted for situations like we have here: a variety of binary features.